In [7]:
!pip3 install pulp


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip3 install --upgrade pip


In [9]:
import pulp

In [131]:
shipping_problem = pulp.LpProblem(name = "shipping_problem", sense = pulp.LpMinimize)
MY_DESTINATIONS = {
    0: "Mombasa",
    1: "Zanzibar",
    2: "Victoria",
    3: "Salalah",
    4: "Mombasa",
}
PRICES = [600, 900, 1100, 1200, 600]
FUEL_USAGE = [700, 1800, 2200, 2800]
MINIMUM_TANK_AMOUNT = [1000, 1200, 1800, 2800, 1000]
MAXIMUM_TANK_AMOUNT = [7000, 7000, 7000, 7000, 7000]
TANK_ONE = [0, 0, 0, 0, 0]
TANK_TWO = [0, 0, 0, 0, 0]
FIXED_COSTS = [2000000, 1000000, 4000000, 3000000, 0]

SAILING_DAYS = [0, 2, 6, 6, 7]

# Variables
s1 = pulp.LpVariable.dict(name = "s1", indices=MY_DESTINATIONS.keys(), lowBound = 0, cat = "Integer", upBound = 3500)
s2 = pulp.LpVariable.dict(name = "s2", indices=MY_DESTINATIONS.keys(), lowBound = 0, cat = "Integer", upBound = 3500)
h_xs = pulp.LpVariable.dict(name = "h_xs", indices=MY_DESTINATIONS.keys(), lowBound = 0, cat = "Integer", upBound = 7000)
# Objective function
shipping_problem += sum((FIXED_COSTS[i] +(PRICES[i] * (s1[i] + s2[i]))) for i in range(len(PRICES))), "Objective Function"

# Constraints
# Eq 5 and 1
for i in range(len(FUEL_USAGE)):
    # Eq 2
    if i == 0:
        shipping_problem += h_xs[i] == (s1[i] + s2[i]) * (1+SAILING_DAYS[i]*0.01), f"Eq2 for {MY_DESTINATIONS[i]}"
    else:
        shipping_problem += h_xs[i] == h_xs[i-1] + (s1[i-1] + s2[i-1])  - FUEL_USAGE[i-1], f"Eq 2 for {MY_DESTINATIONS[i]}"
    shipping_problem += s1[i] + s2[i] >= MINIMUM_TANK_AMOUNT[i], f"Minimum tank amount constraint for {MY_DESTINATIONS[i]}"
    shipping_problem += s1[i] + s2[i] <= MAXIMUM_TANK_AMOUNT[i], f"Maximum tank amount constraint for {MY_DESTINATIONS[i]}"
    # Eq 1
    shipping_problem += s1[i] + s2[i] <= h_xs, f"Eq 1 for {MY_DESTINATIONS[i]}"
    # Eq 4
    shipping_problem += h_xs[i] >= MINIMUM_TANK_AMOUNT[i] + FUEL_USAGE[i] * 3/2, f"Eq 4 for {MY_DESTINATIONS[i]}"
    # Problem 14
    shipping_problem += s1[i] == s2[i], f"Problem 14 for {MY_DESTINATIONS[i]}"
    s1[i] *= (1-SAILING_DAYS[i]*0.01)
    s2[i] *= (1-SAILING_DAYS[i]*0.01)

# Solve the problem
shipping_problem += s1[3] + s2[3] >= 1000 + FUEL_USAGE[3]
status = shipping_problem.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/fp/6fd8hwdx18z6pbm4tp805qzh0000gn/T/154604fc5d134d2190b7ac0e595c6ec3-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/fp/6fd8hwdx18z6pbm4tp805qzh0000gn/T/154604fc5d134d2190b7ac0e595c6ec3-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 30 COLUMNS
At line 144 RHS
At line 170 BOUNDS
At line 186 ENDATA
Problem MODEL has 25 rows, 15 columns and 73 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1.13886e+07 - 0.00 seconds
Cgl0003I 0 fixed, 8 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 8 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 7 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0003

In [92]:
for i in range(len(FUEL_USAGE)):
  print(f"| h[{i}] = {h_xs[i].value()} | f[{i}] = {FUEL_USAGE[i]} | l[1,{i}] = {s1[i].value()} | l[2,{i}] = {s2[i].value()} | \n")


| h[0] = 3650.0 | f[0] = 700 | l[1,0] = 1825.0 | l[2,0] = 1825.0 | 

| h[1] = 6600.0 | f[1] = 1800 | l[1,1] = 600.0 | l[2,1] = 600.0 | 

| h[2] = 6000.0 | f[2] = 2200 | l[1,2] = 900.0 | l[2,2] = 900.0 | 

| h[3] = 5600.0 | f[3] = 2800 | l[1,3] = 1900.0 | l[2,3] = 1900.0 | 



In [50]:
MY_DESTINATIONS[(4+1) % 4]

'Zanzibar'